# 🚀 Pipeline Unificado de Treinamento

Este notebook demonstra o uso do **pipeline unificado** que suporta tanto **classificação** quanto **regressão** através de uma única interface.

## 🎯 Principais Características

- ✅ **Um único pipeline** para classificação e regressão
- ✅ **Mesma API** para ambos os tipos
- ✅ **Métricas automáticas** selecionadas por tipo
- ✅ **Fácil de usar** - apenas 1 parâmetro diferente!

---

## 📦 Setup

In [1]:
import sys
sys.path.append('..')

import pandas as pd
import numpy as np
from sklearn.datasets import make_classification, make_regression
from src.pipelines import treinar_pipeline_completo, treinar_rapido

print("✓ Imports OK")

KeyboardInterrupt: 

## 📊 Preparar Dados

Vamos criar dados sintéticos para demonstração:

In [ ]:
# Dados de CLASSIFICAÇÃO
X_clf, y_clf = make_classification(
    n_samples=500,
    n_features=20,
    n_informative=10,
    n_redundant=5,
    random_state=42
)
df_classificacao = pd.DataFrame(X_clf, columns=[f'feature_{i}' for i in range(20)])
df_classificacao['classe'] = y_clf

print(f"Dados de classificação: {df_classificacao.shape}")
print(f"Classes: {df_classificacao['classe'].value_counts().to_dict()}")

# Dados de REGRESSÃO
X_reg, y_reg = make_regression(
    n_samples=500,
    n_features=20,
    n_informative=10,
    random_state=42
)
df_regressao = pd.DataFrame(X_reg, columns=[f'feature_{i}' for i in range(20)])
df_regressao['valor'] = y_reg

print(f"\nDados de regressão: {df_regressao.shape}")
print(f"Valor (stats): {df_regressao['valor'].describe()}")

---

## 🎯 EXEMPLO 1: Classificação - Treinamento Completo

Pipeline completo com todas as etapas:
1. Setup
2. Comparação de modelos
3. Otimização
4. Finalização
5. Salvamento

In [ ]:
resultado_clf = treinar_pipeline_completo(
    dados=df_classificacao,
    coluna_alvo='classe',
    tipo_problema='classificacao',  # ← Define o tipo!
    n_modelos_comparar=3,
    metrica_ordenacao='Accuracy',  # Padrão para classificação
    otimizar_hiperparametros=True,
    n_iter_otimizacao=10,
    finalizar=True,
    salvar_modelo_final=False  # True para salvar
)

In [ ]:
# Ver resultados
print("🏆 MELHOR MODELO:", type(resultado_clf['melhor_modelo']).__name__)
print("\n📊 MÉTRICAS:")
for metrica, valor in resultado_clf['metricas_melhor'].items():
    if isinstance(valor, (int, float)):
        print(f"  {metrica}: {valor:.4f}")

print("\n📈 COMPARAÇÃO DE MODELOS:")
resultado_clf['tabela_comparacao'][['Accuracy', 'AUC', 'F1']].head()

---

## 📈 EXEMPLO 2: Regressão - Treinamento Completo

**Mesma função, apenas muda o `tipo_problema`!**

In [ ]:
resultado_reg = treinar_pipeline_completo(
    dados=df_regressao,
    coluna_alvo='valor',
    tipo_problema='regressao',  # ← Única diferença!
    n_modelos_comparar=3,
    metrica_ordenacao='R2',  # Padrão para regressão
    otimizar_hiperparametros=True,
    n_iter_otimizacao=10,
    finalizar=True,
    salvar_modelo_final=False
)

In [ ]:
# Ver resultados
print("🏆 MELHOR MODELO:", type(resultado_reg['melhor_modelo']).__name__)
print("\n📊 MÉTRICAS:")
for metrica, valor in resultado_reg['metricas_melhor'].items():
    if isinstance(valor, (int, float)):
        print(f"  {metrica}: {valor:.4f}")

print("\n📈 COMPARAÇÃO DE MODELOS:")
resultado_reg['tabela_comparacao'][['MAE', 'MSE', 'R2']].head()

---

## ⚡ EXEMPLO 3: Treinamento Rápido

Para prototipagem e testes rápidos:

In [ ]:
# Classificação rápida
exp_clf_rapido, modelo_clf_rapido = treinar_rapido(
    dados=df_classificacao,
    coluna_alvo='classe',
    tipo_problema='classificacao',
    modelo='rf',  # ou 'auto' para deixar PyCaret escolher
    salvar=False
)

print("✓ Modelo de classificação rápido:", type(modelo_clf_rapido).__name__)

In [ ]:
# Regressão rápida
exp_reg_rapido, modelo_reg_rapido = treinar_rapido(
    dados=df_regressao,
    coluna_alvo='valor',
    tipo_problema='regressao',
    modelo='auto',  # Deixa PyCaret escolher o melhor
    salvar=False
)

print("✓ Modelo de regressão rápido:", type(modelo_reg_rapido).__name__)

---

## 🎯 EXEMPLO 4: Modelos Específicos

Treinar apenas modelos específicos:

In [ ]:
resultado_especifico = treinar_pipeline_completo(
    dados=df_classificacao,
    coluna_alvo='classe',
    tipo_problema='classificacao',
    modelos_incluir=['rf', 'lr', 'dt'],  # Apenas estes 3
    n_modelos_comparar=1,  # Seleciona apenas o melhor
    otimizar_hiperparametros=False,
    finalizar=False,
    salvar_modelo_final=False
)

print("Modelos testados:")
print(resultado_especifico['tabela_comparacao'][['Accuracy', 'F1']])

---

## 📊 EXEMPLO 5: Diferentes Métricas de Ordenação

In [ ]:
# Classificação ordenada por F1
resultado_f1 = treinar_pipeline_completo(
    dados=df_classificacao,
    coluna_alvo='classe',
    tipo_problema='classificacao',
    n_modelos_comparar=2,
    metrica_ordenacao='F1',  # Ordena por F1 em vez de Accuracy
    otimizar_hiperparametros=False,
    finalizar=False,
    salvar_modelo_final=False
)

print("Melhores modelos por F1:")
print(resultado_f1['tabela_comparacao'][['F1', 'Accuracy', 'AUC']].head())

In [ ]:
# Regressão ordenada por MAE
resultado_mae = treinar_pipeline_completo(
    dados=df_regressao,
    coluna_alvo='valor',
    tipo_problema='regressao',
    n_modelos_comparar=2,
    metrica_ordenacao='MAE',  # Ordena por MAE em vez de R2
    otimizar_hiperparametros=False,
    finalizar=False,
    salvar_modelo_final=False
)

print("\nMelhores modelos por MAE:")
print(resultado_mae['tabela_comparacao'][['MAE', 'RMSE', 'R2']].head())

---

## 🎁 Resumo dos Benefícios

| Benefício | Descrição |
|-----------|----------|
| **Unificado** | Um único pipeline para classificação e regressão |
| **Simples** | Apenas 1 parâmetro (`tipo_problema`) define o tipo |
| **Automático** | Métricas selecionadas automaticamente |
| **Flexível** | Customizável em todos os aspectos |
| **Consistente** | Mesma API para ambos os tipos |
| **Manutenível** | Sem duplicação de código |

---

## 📚 Documentação Completa

- **README:** `src/pipelines/README_UNIFIED.md`
- **Exemplos:** `exemplos/exemplo_pipeline_unificado.py`
- **Testes:** `tests/test_pipeline_unificado.py`
- **Refatoração:** `docs/refatoracao-pipeline-unificado.md`

---

## 🚀 Próximos Passos

1. Teste com seus próprios dados
2. Experimente diferentes modelos
3. Ajuste hiperparâmetros
4. Salve e faça deploy dos modelos

**Bom treinamento! 🎉**